In [15]:
import pandas as pd
import numpy as np
import os

# Load the data
try:
    orders = pd.read_csv('../data/olist_orders_dataset.csv')
    items = pd.read_csv('../data/olist_order_items_dataset.csv')
    products = pd.read_csv('../data/olist_products_dataset.csv')
    translations = pd.read_csv('../data/product_category_name_translation.csv') # Original category names are in Portuguese

    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: Files not found.")

Data loaded successfully!


In [16]:
# The Big Merge

# Join Orders with Items
df = pd.merge(orders, items, on='order_id', how='inner')

# Join with Products
df = pd.merge(df, products, on='product_id', how='inner')

# Join with Translations
df = pd.merge(df, translations, on='product_category_name', how='inner')

print(f"Merged Dataset Shape: {df.shape}")

Merged Dataset Shape: (111023, 23)


In [17]:
# Feature Engineering

# Convert timestamp columns
time_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date', 
             'order_estimated_delivery_date']

for col in time_cols:
    df[col] = pd.to_datetime(df[col])

# Feature 1: Actual Lead Time (Days)
df['actual_lead_time_days'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

# Feature 2: Estimated vs Actual Diff
df['delivery_performance'] = (df['order_estimated_delivery_date'] - df['order_delivered_customer_date']).dt.days

print("Dates converted and Lead Times calculated.")

Dates converted and Lead Times calculated.


In [18]:
# Cleanup

# Filter only delivered orders
df_delivered = df[df['order_status'] == 'delivered'].copy()

# Drop rows where lead time is null
df_delivered.dropna(subset=['actual_lead_time_days'], inplace=True)

# Select columns for master table
master_table = df_delivered[[
    'order_id', 'product_id', 'product_category_name_english', 
    'price', 'freight_value', 'order_purchase_timestamp', 
    'actual_lead_time_days', 'delivery_performance'
]]

# Rename the column
master_table = master_table.rename(columns={'product_category_name_english': 'category'})

print("Master table created!")
print(f"Total Records: {len(master_table)}")
display(master_table.head())

Master table created!
Total Records: 108630


,order_id,product_id,category,price,freight_value,order_purchase_timestamp,actual_lead_time_days,delivery_performance
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,housewares,29.99,8.72,2017-10-02 10:56:33,8.0,7.0
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,perfumery,118.70,22.76,2018-07-24 20:41:37,13.0,5.0
2,47770eb9100c2d0c44946d9cf07ec65d,aa4383b373c6aca5d8797843e5594415,auto,159.90,19.22,2018-08-08 08:38:49,9.0,17.0
3,949d5b44dbf5de918fe9c16f97b45f8a,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop,45.00,27.20,2017-11-18 19:28:06,13.0,12.0
4,ad21c59c0840e6cb83a9ceb5573f8159,65266b2da20d04dbe00c5c2d3bb7859e,stationery,19.90,8.72,2018-02-13 21:18:39,2.0,9.0


In [19]:
# Save the master table
master_table.to_csv('../data/master_table.csv', index=False)

print("File saved successfully: master_table_clean.csv")

File saved successfully: master_table_clean.csv
